In [ ]:
from tensorflow.keras import layers, Model, preprocessing
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.optimizers import Adam
import numpy as np

from utils import make_predictions, evaluation, data_loader

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
train_ds, val_ds, test_ds, class_names, image_size = data_loader.load_data(RANDOM_SEED)

In [ ]:
# efficientnet base model

efficientNetBaseArchitecture = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size[0], image_size[1], 3),
    pooling='avg'
)

# Freezing the early layers (retain general features)
efficientNetBaseArchitecture.trainable = True
for layer in efficientNetBaseArchitecture.layers[:-20]:  # freeze all but last 20 layers
    layer.trainable = False

# Custom classification head
x = efficientNetBaseArchitecture.output
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(len(class_names), activation='softmax')(x)

efficientNet = Model(inputs=efficientNetBaseArchitecture.input, outputs=predictions)

efficientNet.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

efficientNet.summary()

In [ ]:
history = efficientNet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

In [ ]:
evaluation.plot_loss(history)
evaluation.plot_accuracy(history)

In [ ]:
y_true, y_pred = make_predictions.predict_test_data(efficientNet, test_ds)

In [ ]:
evaluation.make_classification_report(y_true, y_pred, class_names)

In [ ]:
evaluation.make_confusion_matrix(y_true, y_pred, class_names)